In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pydot
import os
import matplotlib.pyplot as plt
import copy
import time
from collections import Counter
import scene
import AllegroWrapper
import contact
import motion_planning
import search
import visualizations
from pydrake.all import StartMeshcat, RigidTransform, RotationMatrix
from manipulation.utils import ConfigureParser

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
meshcat = None

In [34]:
p_W_brick = [0.65, -0.28, 0.015] #[np.random.uniform(0.4, 0.75), np.random.uniform(-0.35, 0.35), 0.01]
R_W_brick = [0, 0, 274]
meshcat, station, simulator, context, p_W_brick, R_W_brick = scene.init_scenario(brick_location=p_W_brick, brick_rotation=R_W_brick, meshcat=meshcat)

allegro = AllegroWrapper.AllegroHand(station, context)
simulator.set_target_realtime_rate(1.0)
x0 = station.GetOutputPort("iiwa+allegro.state_estimated").Eval(context)
# scene.render_station_diagram(station)

def reset_arm(station, simulator, context, x0):
    station.GetInputPort("iiwa+allegro.desired_state").FixValue(context, x0)
    simulator.AdvanceTo(context.get_time() + 1)

reset_arm(station, simulator, context, x0)

In [35]:
def investigate(simulator, station, p_W_obj, all_touched_points):
    """
    Given contact with an object, sample a number of contact locations nearby and 
    plan a trajectory with a finger to the object, in order to build up a point cloud
    of the object for pose estimation
    """
    context = simulator.get_mutable_context()   
    plant = station.GetSubsystemByName("plant")
    gripper = plant.GetBodyByName("hand_root") # tip of the index finger
    plant_context = plant.GetMyContextFromRoot(context)

    def raise_hand(amount):
        iiwa_body = plant.GetBodyByName("hand_root")
        X_WG = plant.EvalBodyPoseInWorld(plant_context, iiwa_body)

        start_pos = np.array(X_WG.translation())
        end_pos = start_pos + np.array([0, 0, amount])
        print(start_pos, end_pos)
        # Linear interpolation in Z-axis
        interp_positions = np.linspace(start_pos, end_pos, 5)
        print(interp_positions)
        # Generate list of RigidTransform objects
        pose_list = [RigidTransform(X_WG.rotation(), pos) for pos in interp_positions]

        obj_touched, new_contacts, p_W_obj = motion_planning.move_arm(end_pos, 
                                                simulator, 
                                                station, 
                                                context, 
                                                time_interval=0.4, 
                                                poses=pose_list,
                                                frame="hand_root", 
                                                arc_height=0.2,
                                                stop_on_contact=False,
                                                freeze_fingers=False,
                                                position_tol=np.array([0.1, 0.1, 0.01])
                                                )
        print("Raised arm up!")
        return None

    raise_hand(0.5)
    simulator.AdvanceTo(context.get_time() + 1)

    num_samples = 20
    nearby_x = np.random.uniform(p_W_obj[0] - 0.09, p_W_obj[0] + 0.09, num_samples)
    nearby_y = np.random.uniform(p_W_obj[1] - 0.09, p_W_obj[1] + 0.09, num_samples)
    nearby_z = np.ones(num_samples) * p_W_obj[2]

    nearby_points = np.vstack((nearby_x, nearby_y, nearby_z)).T
    for point in nearby_points:
        obj_touched, new_contacts, p_W_obj = motion_planning.move_arm(point, 
                                                simulator, 
                                                station, 
                                                context, 
                                                time_interval=0.4, 
                                                frame="link_2", 
                                                arc_height=0.5,
                                                stop_on_contact=True,
                                                freeze_fingers=False,
                                                state_update_len=23,
                                                simulator_steps=1000,
                                                position_tol=[0.01, 0.01, 0.01])
        for contact in new_contacts:
            all_touched_points.add(tuple(contact))   

        # raise_hand(0.5)
        simulator.AdvanceTo(context.get_time() + .25)
        
    return all_touched_points

In [36]:
def run_search(station, frame = "hand_root", object_bias=None):
    reset_arm(station, simulator, context, x0)
    plant = station.GetSubsystemByName("plant")
    all_touched_points = set()

    # Get initial pose of the gripper by using default context of manip station.
    gripper = plant.GetBodyByName(frame)
    plant_context = plant.GetMyContextFromRoot(context)
    
    X_WG = plant.EvalBodyPoseInWorld(plant_context, gripper)
    table_end_effector_poses = contact.get_table_contact(X_WG)
    p_WG_post = table_end_effector_poses[-1].translation()
       
    obj_touched, new_contacts, p_W_obj = motion_planning.move_arm(p_WG_post, simulator, station, context, time_interval=0.4, frame=frame)
    touch_history = []

    if obj_touched == "table":
        table_height = min(new_contacts[:, 2])
        touched_point = new_contacts[np.random.choice(np.arange(len(new_contacts)))]
        print("Table height: ", table_height)
    else:
        print("Table not found")

    touch_history.append(touched_point[:2])
    for point in new_contacts:
        all_touched_points.add(tuple(point))

    i = 0
    while obj_touched != "object":
        if i > 30:
            break
        i += 1
        
        next_touch = search.sample_new_target(touch_history, bias_position=object_bias, object_radius=0.03)
        next_point = [next_touch[0], next_touch[1], table_height]
        touch_history.append(next_touch)
        
        #next_point = p_W_brick
        X_WG = plant.EvalBodyPoseInWorld(plant_context, gripper)

        # NOTE: This can change if there's biases in the search algorithm, based on the object's height and expected distance from the gripper
        # NOTE: We might also modify it based on some energy constraint, to minimize movement costs
        obj_touched, new_contacts, p_W_obj = motion_planning.move_arm(next_point, 
                                                simulator, 
                                                station, 
                                                context, 
                                                time_interval=0.4, 
                                                frame=frame, 
                                                stop_on_contact=True,
                                                arc_height=0.2)
                                                
        for point in new_contacts:
            all_touched_points.add(tuple(point))

        X_WG = plant.EvalBodyPoseInWorld(plant_context, gripper)
        #fig = visualizations.plot_2d_search(X_WG.translation(), all_touched_points, next_touch, object_bias, p_W_brick)
        #plt.show()

    print("Object found! Location at: ", p_W_obj)
    print("Actual object location: ", p_W_brick)
    print("Difference: ", np.linalg.norm(np.array(p_W_obj)[:2] - np.array(p_W_brick[:2])))
    return all_touched_points, p_W_obj

all_touched_points, object_contact = run_search(station, object_bias=[p_W_brick[0], p_W_brick[1]])

Table height:  -0.0005481045260718378
Object found! Location at:  [ 0.62674944 -0.24457775  0.04877561]
Actual object location:  [0.65, -0.28, 0.015]
Difference:  0.04237127172170303


In [37]:
# reset_arm(station, simulator, context, x0)
# allegro.set_positions(["thumb_revolute_z"], [1.0])


In [38]:
all_touched_points = investigate(simulator, station, object_contact, all_touched_points)

[ 0.62498233 -0.23765418  0.05899313] [ 0.62498233 -0.23765418  0.55899313]
[[ 0.62498233 -0.23765418  0.05899313]
 [ 0.62498233 -0.23765418  0.18399313]
 [ 0.62498233 -0.23765418  0.30899313]
 [ 0.62498233 -0.23765418  0.43399313]
 [ 0.62498233 -0.23765418  0.55899313]]
Raised arm up!


In [39]:
p_WC = np.array(list(all_touched_points))
p_WC[np.where(p_WC[:, 2] > 0.001)]

array([[ 0.69701723, -0.27842136,  0.06327184],
       [ 0.65827696, -0.2839196 ,  0.07849263],
       [ 0.67132387, -0.28019479,  0.07147382],
       [ 0.68530324, -0.32409871,  0.0813527 ],
       [ 0.65949932, -0.28399281,  0.07896814],
       [ 0.68233855, -0.32318067,  0.08124733],
       [ 0.68281905, -0.32348685,  0.08124943],
       [ 0.62549583, -0.24631882,  0.04892221],
       [ 0.67886597, -0.31829935,  0.08080986],
       [ 0.68574313, -0.32422258,  0.08134285],
       [ 0.67721091, -0.31522571,  0.04898747],
       [ 0.67200355, -0.28661946,  0.07431571],
       [ 0.67332543, -0.28193237,  0.07247922],
       [ 0.68023408, -0.32046414,  0.08137637],
       [ 0.6729185 , -0.28148934,  0.07226078],
       [ 0.66209193, -0.28423621,  0.07987392],
       [ 0.68071331, -0.32103219,  0.08142343],
       [ 0.68158031, -0.32207065,  0.08136805],
       [ 0.68197491, -0.32260996,  0.08131109],
       [ 0.67754159, -0.31375945,  0.0799897 ],
       [ 0.66008141, -0.28403112,  0.079

In [40]:
pc = np.array(list(all_touched_points))
box_points = pc[np.where(pc[:, 2] > 0.0001)]
print(box_points)
plt.scatter(box_points[:, 0], box_points[:, 1])
plt.scatter(p_W_brick[0], p_W_brick[1], c="r", marker="x")
plt.xlim(-1, 1)
plt.ylim(-1, 1)

[[ 0.69701723 -0.27842136  0.06327184]
 [ 0.65827696 -0.2839196   0.07849263]
 [ 0.67132387 -0.28019479  0.07147382]
 [ 0.68530324 -0.32409871  0.0813527 ]
 [ 0.65949932 -0.28399281  0.07896814]
 [ 0.68233855 -0.32318067  0.08124733]
 [ 0.68281905 -0.32348685  0.08124943]
 [ 0.62549583 -0.24631882  0.04892221]
 [ 0.67886597 -0.31829935  0.08080986]
 [ 0.68574313 -0.32422258  0.08134285]
 [ 0.67721091 -0.31522571  0.04898747]
 [ 0.67200355 -0.28661946  0.07431571]
 [ 0.67332543 -0.28193237  0.07247922]
 [ 0.68023408 -0.32046414  0.08137637]
 [ 0.6729185  -0.28148934  0.07226078]
 [ 0.66209193 -0.28423621  0.07987392]
 [ 0.68071331 -0.32103219  0.08142343]
 [ 0.68158031 -0.32207065  0.08136805]
 [ 0.68197491 -0.32260996  0.08131109]
 [ 0.67754159 -0.31375945  0.0799897 ]
 [ 0.66008141 -0.28403112  0.07918346]
 [ 0.67240386 -0.28106602  0.07205746]
 [ 0.68387737 -0.32372804  0.08132465]
 [ 0.66062073 -0.28407676  0.07937799]
 [ 0.6833583  -0.3236065   0.0812923 ]
 [ 0.68115562 -0.32155572

(-1.0, 1.0)

In [41]:
# generate 3D scatterplot of pointcloud + pointcloud of actual brick
import matplotlib
matplotlib.use('Qt5Agg')

brick_size = [0.075, 0.05, 0.05]

def generate_brick_pointcloud(p_W_brick, R_W_brick_deg, brick_size, points_per_side=10):
    """
    Returns a pointcloud of a brick in world coordinates.
    
    p_W_brick: np.array: (3,) array with center of brick coordinates (in world coordinates)
    R_W_brick: np.ndarray: (4, 4) rotation matrix for brick (in world coordinates)
    brick_size: np.array: (3,) array with x, y, and z size
    """
    # Generate a grid of points in the local frame of the brick
    # Convert rotation from degrees to radians
    rotation_radians = np.radians(R_W_brick_deg[2])

    # Create rotation matrix for z-axis rotation
    R_z = np.array([
        [np.cos(rotation_radians), -np.sin(rotation_radians), 0],
        [np.sin(rotation_radians), np.cos(rotation_radians), 0],
        [0, 0, 1]
    ])

    # Generate a grid of points in the local frame of the brick
    x = np.linspace(-brick_size[0]/2, brick_size[0]/2, points_per_side)
    y = np.linspace(-brick_size[1]/2, brick_size[1]/2, points_per_side)
    z = np.linspace(-brick_size[2]/2, brick_size[2]/2, points_per_side)
    X, Y, Z = np.meshgrid(x, y, z)

    # Flatten the grid to a list of points
    points_local = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T

    # Apply rotation and translation to transform points to the world frame
    points_world = np.dot(points_local, R_z.T) + p_W_brick

    return points_world

brick_points = generate_brick_pointcloud(p_W_brick, R_W_brick, brick_size)

def visualize_pointcloud_results(box_points, brick_points):
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Scatter plot for the brick point cloud
    ax.scatter(brick_points[:, 0], brick_points[:, 1], brick_points[:, 2], c='r', marker='o', label='Brick Point Cloud')

    # Scatter plot for the robot hand point cloud
    ax.scatter(box_points[:, 0], box_points[:, 1], box_points[:, 2], c='b', marker='^', label='Estimated Point Cloud', s=100)

    
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')

visualize_pointcloud_results(box_points, brick_points)
plt.show()


In [110]:
def fit_plane_to_points(points):
    """
    Fits a plane to a set of points using PCA and returns the plane's normal.

    Args:
        points (np.array): Nx3 array of 3D points.

    Returns:
        normal (np.array): The normal vector of the fitted plane.
    """

    centroid = np.mean(points, axis=0)
    centered_points = points - centroid
    covariance_matrix = np.cov(centered_points.T)
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

    # The normal of the plane is the eigenvector corresponding to the smallest eigenvalue
    sorted_indices = np.argsort(np.abs(eigenvalues)) 
    max_eigenvectors = eigenvectors[:, sorted_indices[-2:]]

    return max_eigenvectors

In [111]:
x_est_W = np.mean(box_points[:, 0])
y_est_W = np.mean(box_points[:, 1])
z_est_W = np.mean(box_points[:, 2])
print("Estimated object location: ", x_est_W, y_est_W, z_est_W)

# Estimate Orientation by fitting a plane to the points
orientation = fit_plane_to_points(box_points).T
print(orientation.shape)
x1, y1 = orientation[0][:2]
x2, y2 = orientation[1][:2]
angle = np.rad2deg(np.arctan((y2 - y1) / (x2 - x1))) + 180

print("Estimated angle: ", angle)


Estimated object location:  0.6739495312234425 -0.29392092742599857 0.04883863236777456
(2, 3)
Estimated angle:  138.2922184020169


In [112]:
print("Actual position and Angle: ", p_W_brick, R_W_brick)

Actual position and Angle:  [0.65, -0.28, 0.015] [0, 0, 274]


In [113]:
# allegro.close_hand()
# simulator.AdvanceTo(context.get_time() + 1)
                                            

In [114]:
# allegro.open_hand()
# simulator.AdvanceTo(context.get_time() + 1)


In [115]:
frame = "hand_root"
plant = station.GetSubsystemByName("plant")
plant_context = plant.GetMyContextFromRoot(context)
world_frame = plant.world_frame()
X_WG = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName(frame))


In [116]:
def raise_hand(amount):
        iiwa_body = plant.GetBodyByName("hand_root")
        X_WG = plant.EvalBodyPoseInWorld(plant_context, iiwa_body)

        start_pos = np.array(X_WG.translation())
        end_pos = start_pos + np.array([0, 0, amount])
        print(start_pos, end_pos)
        # Linear interpolation in Z-axis
        interp_positions = np.linspace(start_pos, end_pos, 5)
        print(interp_positions)
        # Generate list of RigidTransform objects
        pose_list = [RigidTransform(X_WG.rotation(), pos) for pos in interp_positions]

        obj_touched, new_contacts, p_W_obj = motion_planning.move_arm(end_pos, 
                                                simulator, 
                                                station, 
                                                context, 
                                                time_interval=0.4, 
                                                poses=pose_list,
                                                frame="hand_root", 
                                                arc_height=0.2,
                                                stop_on_contact=False,
                                                freeze_fingers=False,
                                                position_tol=np.array([0.1, 0.1, 0.01])
                                                )

In [117]:
raise_hand(0.2)

# Open hand
allegro.open_hand()
simulator.AdvanceTo(context.get_time() + .5)
allegro.set_positions(["thumb_revolute_z", "thumb_revolute_y", "thumb_1", "thumb_2"], [1.5, .9, 1.2, 0.5])
simulator.AdvanceTo(context.get_time() + .5)


# Position Hand in pre-grasp mode
R_WG = X_WG.rotation()
X_Wbrick_estimated = RigidTransform(RotationMatrix().MakeZRotation(angle), [x_est_W, y_est_W, z_est_W])

X_BG = X_Wbrick_estimated.inverse().multiply(copy.deepcopy(X_WG))
offset_BGripper = np.array([-0.05, 0.0, 0.05])
p_BG_pre = X_BG.translation() + offset_BGripper
R_BG_pre = X_BG.rotation()

X_BG_pre = RigidTransform(R_BG_pre, p_BG_pre)
X_WG_pre = X_Wbrick_estimated.multiply(X_BG_pre)

pose_list = [X_WG_pre]

_, _, _ = motion_planning.move_arm(None, 
                                    simulator, 
                                    station, 
                                    context, 
                                    poses=pose_list,
                                    time_interval=1, 
                                    frame=frame, 
                                    stop_on_contact=False,
                                    arc_height=0.2)


# Move hand down
X_WG_pick = copy.deepcopy(X_WG_pre)
X_WG_pick.set_translation(X_WG_pick.translation() + np.array([0, 0, -0.1]))
pose_list = [X_WG_pick]

simulator.AdvanceTo(context.get_time() + 1)
_, _, _ = motion_planning.move_arm(None, 
                                    simulator, 
                                    station, 
                                    context, 
                                    poses=pose_list,
                                    time_interval=1, 
                                    frame=frame, 
                                    arc_height=0.2
                                )
simulator.AdvanceTo(context.get_time() + 1)


[ 0.51915296 -0.32727821  0.13005945] [ 0.51915296 -0.32727821  0.33005945]
[[ 0.51915296 -0.32727821  0.13005945]
 [ 0.51915296 -0.32727821  0.18005945]
 [ 0.51915296 -0.32727821  0.23005945]
 [ 0.51915296 -0.32727821  0.28005945]
 [ 0.51915296 -0.32727821  0.33005945]]


In [118]:

# Close hand
simulator.AdvanceTo(context.get_time() + 1)
allegro.close_hand()
simulator.AdvanceTo(context.get_time() + 1)
allegro.tighten_hand()
simulator.AdvanceTo(context.get_time() + 1)
X_WG_post = copy.deepcopy(X_WG_pick)
X_WG_post.set_translation(X_WG_post.translation() + np.array([0, 0, 0.05]))
pose_list = [X_WG_post]

# allegro.tighten_hand()
# simulator.AdvanceTo(context.get_time() + 1)

# Try and pick the fucker up
_, _, _ = motion_planning.move_arm(None, 
                                    simulator, 
                                    station, 
                                    context, 
                                    poses=pose_list,
                                    time_interval=1, 
                                    frame=frame, 
                                    arc_height=0.2)
simulator.AdvanceTo(context.get_time() + 1)                                   